In [1]:
# Imports
from IPython.display import HTML          
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_metric, Dataset
from sklearn.model_selection import train_test_split

In [2]:
# Left align data in pd tables
HTML("""
<style>
    th, td {
        text-align: left !important;
    }
</style>
""")

In [3]:
# Import data and clean 
# Data is from https://www.kaggle.com/datasets/arshkandroo/behavioural-tweets?select=Lonely_Tweets.csv.
# Tweepy API was used to scrape twitter for lonely and normal tweets. The quality is okay but 
# some misclassification. The tweets are already cleaned using the NLTK library according to the
# data collectors.

df_normal = pd.read_csv('Data/Normal_Tweets.csv.xls')
del df_normal['Unnamed: 0']
df_normal.columns.values[0] = 'tweet'
df_normal['label'] = 0                                  # 0 for normal tweets 
df_normal['id'] = range(1, len(df_normal) + 1)

df_lonely = pd.read_csv('Data/Lonely_Tweets.csv.xls')
del df_lonely['Unnamed: 0']
df_lonely.columns.values[0] = 'tweet'
df_lonely['label'] = 1                                  # 1 for lonely tweets
df_lonely['id'] = range(len(df_normal) + 1, len(df_normal) + len(df_lonely) + 1)

df = pd.concat([df_normal, df_lonely])

In [4]:
# Display summary data
print(f"The data set has {len(df)} tweets in it.")
print()

print(f"There are {df.shape[1]} variables and they are as follows: ")
print(df.dtypes)
print()

print("The first and last five rows of the table are:")
pd.concat([df.head(), df.tail()])


The data set has 18447 tweets in it.

There are 3 variables and they are as follows: 
tweet    object
label     int64
id        int64
dtype: object

The first and last five rows of the table are:


,tweet,label,id
0,remember hillary email non secure server,0,1
1,cant avoid demon,0,2
2,por fin la pusieron en spotify losing way de f...,0,3
3,kills,0,4
4,thank introduce important gunsense law make co...,0,5
8517,love son get paternity test person sex doesnt ...,1,18443
8518,tell father dont want bother need talk god cau...,1,18444
8519,lrt leave alone finally found need cry love ha...,1,18445
8520,know something dont need one love want want wa...,1,18446
8521,dont need want meet want several moment side,1,18447


In [5]:
# Split into training and validation set
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
train_texts = train_df.tweet.tolist()
train_labels = train_df.label.tolist()

labels = torch.tensor(train_labels)

# Vectorize data in one step using the tokenizer
encoded_inputs = tokenizer(
    train_texts,
    add_special_tokens=True,
    truncation=True,
    max_length=20,
    padding='max_length',
    return_attention_mask=True,
    return_tensors="pt",
)

input_ids = encoded_inputs["input_ids"]
attention_masks = encoded_inputs["attention_mask"]


In [8]:
train_data_dict = {
    'input_ids': input_ids,
    'attention_mask': attention_masks,
    'labels': labels
}

# # Move each tensor in the dictionary to the GPU
# train_data_dict = {key: tensor.to("mps") for key, tensor in train_data_dict.items()}

# # Verify each tensor's device
# for key, tensor in train_data_dict.items():
#     print(f"{key} is on {tensor.device}")

# Create a Dataset object
train_dataset = Dataset.from_dict(train_data_dict)

In [9]:
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# model.to(device)
# print(next(model.parameters()).device)

In [10]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=0.3,
    per_device_train_batch_size=32,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,  # The model to train
    args=training_args,
    train_dataset=train_dataset,  # Your training dataset
    # Optionally add a validation dataset
    # eval_dataset=validation_dataset,
)

In [11]:
trainer.train()
# torch.mps.empty_cache()
# model.to("cpu")
print("hello")

  0%|          | 0/139 [00:00<?, ?it/s]

{'loss': 0.7017, 'grad_norm': 2.767789602279663, 'learning_rate': 4.640287769784173e-05, 'epoch': 0.02}
{'loss': 0.6608, 'grad_norm': 3.600064277648926, 'learning_rate': 4.280575539568346e-05, 'epoch': 0.04}
{'loss': 0.5434, 'grad_norm': 6.691036701202393, 'learning_rate': 3.920863309352518e-05, 'epoch': 0.06}
{'loss': 0.4431, 'grad_norm': 5.390189170837402, 'learning_rate': 3.561151079136691e-05, 'epoch': 0.09}
{'loss': 0.3757, 'grad_norm': 5.2418999671936035, 'learning_rate': 3.201438848920863e-05, 'epoch': 0.11}
{'loss': 0.3749, 'grad_norm': 3.9865145683288574, 'learning_rate': 2.841726618705036e-05, 'epoch': 0.13}
{'loss': 0.3229, 'grad_norm': 5.72578763961792, 'learning_rate': 2.482014388489209e-05, 'epoch': 0.15}
{'loss': 0.2577, 'grad_norm': 4.145211219787598, 'learning_rate': 2.1223021582733816e-05, 'epoch': 0.17}
{'loss': 0.3771, 'grad_norm': 8.23632526397705, 'learning_rate': 1.7625899280575538e-05, 'epoch': 0.19}
{'loss': 0.3315, 'grad_norm': 4.620920181274414, 'learning_rat

In [ ]:
model.save_pretrained("bert_model")
tokenizer.save_pretrained("bert_model")

In [ ]:
# model.to("cpu")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
print("Hello")